# テキストデータの前処理

テキストデータを分析する際には、基本的に以下の処理が行われます。

1. 形態素解析
    * 単語分割
    * 品詞付与
    * 原形抽出
2. 構文解析

日本語を例に、`spaCy`と`GiNZA`というライブラリを使って処理の過程を見ていきましょう。

`spaCy`では上の一連の処理をまとめて行ってくれます。

テキストデータとして、オー・ヘンリー（結城浩訳）[『最後の一枚の葉』](https://www.hyuki.com/trans/leaf.html)の冒頭部分を使います。

In [ ]:
text = """
ワシントン・スクエア西にある小地区は、 道路が狂ったように入り組んでおり、 「プレース」と呼ばれる区域に小さく分かれておりました。 この「プレース」は不可思議な角度と曲線を描いており、 一、二回自分自身と交差している通りがあるほどでした。 かつて、ある画家は、この通りが貴重な可能性を持っていることを発見しました。 例えば絵や紙やキャンバスの請求書を手にした取り立て屋を考えてみてください。 取り立て屋は、この道を歩き回ったあげく、 ぐるりと元のところまで戻ってくるに違いありません。 一セントも取り立てることができずにね。
それで、芸術家たちはまもなく、奇妙で古いグリニッチ・ヴィレッジへとやってきました。 そして、北向きの窓と十八世紀の切り妻とオランダ風の屋根裏部屋と安い賃貸料を探してうろついたのです。 やがて、彼らは しろめ製のマグやこんろ付き卓上なべを一、二個、六番街から持ち込み、 「コロニー」を形成することになりました。
ずんぐりした三階建ての煉瓦造りの最上階では、スーとジョンジーがアトリエを持っていました。 「ジョンジー」はジョアンナの愛称です。 スーはメイン州の、ジョンジーはカリフォルニア州の出身でした。 二人は八番街の「デルモニコの店」の定食で出会い、 芸術と、チコリーのサラダと、ビショップ・スリーブの趣味がぴったりだとわかって、 共同のアトリエを持つことになったのでした。
"""

In [ ]:
# ライブラリのインストール（初回のみ）

!pip install spacy ginza ja-ginza
!pip install scikit-learn pandas

In [ ]:
# ライブラリのインポート
import spacy

# 日本語モデルのロード
nlp = spacy.load("ja_ginza")

# 解析
doc = nlp(text)

# 結果の確認
for token in doc:
    print(token)

形態素解析の結果には、語の原形や品詞の情報も含まれます。

In [ ]:
for token in doc:
    print(f"{token}\t{token.lemma_}\t{token.pos_}\t{token.tag_}")

構文解析の結果は、次のように確認できます。

In [ ]:
import pandas as pd

# 解析結果をpandasのDataFrameに入れる

df = pd.DataFrame({
    "text": token.text,
    "lemma_": token.lemma_,
    "pos_": token.pos_,
    "tag_": token.tag_,
    "dep_": token.dep_,
    "children": list(token.children)
} for token in doc)

In [ ]:
df

In [ ]:
# 係り受けの図を表示する

spacy.displacy.render(doc, style="dep")

前処理が完了したところで、データの確認として単語の使用頻度を数えてみましょう。

In [ ]:
from collections import Counter

# 単語の頻度を数える
counter = Counter(token.lemma_ for token in doc)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

句読点や助詞など、意味がなさそうな言葉ばかりです。

より意味がある語を取り出すために、分析対象とする品詞を指定しましょう。具体的には、内容語である名詞、動詞、形容詞（、固有名詞）を指定すればよいでしょう。

In [ ]:
# 分析対象とする品詞の指定
include_pos = ("NOUN", "VERB", "ADJ", "PROPN")

# 再度単語の頻度を数える
counter = Counter(token.lemma_ for token in doc if token.pos_ in include_pos)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

ちょっとよくなりました。でも、「こと」「ある」「いる」などの一般的な名詞や動詞が多いように思えます。

これらを不要語として指定し、除去しましょう。

In [ ]:
# 分析対象とする品詞と不要語（ストップワード）を指定する
include_pos = ("NOUN", "VERB", "ADJ", "PROPN")
stopwords = ("する", "ある", "おる", "ない", "いう", "もの", "こと", "よう", "なる", "ほう", "いる", "くる")

# 再度単語の頻度を数える
counter = Counter(token.lemma_ for token in doc
                  if token.pos_ in include_pos and token.lemma_ not in stopwords)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

ずっと良くなりました。

これは、語の出現頻度を要素としたテキストデータのベクトル表現で、Bag of Wordsと呼ばれます。

## tf-idf

Bag of Wordsは、「多く出現する語ほど重要である」という直観的な考え方に基づいてテキストデータの特徴を表現したベクトルであり、文書分類を含む多くの自然言語処理タスクに有用です。

しかし、いろいろな文書に多く出現すると予測される「わたし」などの言葉は、出現頻度のわりにさほど重要ではないと考えられます。今回の処理のようにこのような言葉を不要語として指定し、分析から除外することもできますが、この点を考慮した単語の重要度の指標として、tf-idfがあります。

tf-idfは、term frequency（単語頻度）-inverse document frequency（逆文書頻度）の略です。具体的には、「ある文書内である単語がどれくらい多い頻度で出現するか」を表すterm frequencyと、「全文書内である単語を含む文書がどれくらい少ない頻度で出現するか」を表すinverse document frequencyをかけ合わせた値です。どの文書にもよく出てくる単語の重要度を下げて、あまり出てこない単語の重要度を上げるために工夫された指標です。

$$
tf-idf = tf（単語頻度） \times \frac{N（文書総数）}{n（単語が出現する文書数）}
$$



tf-idfはヒューリスティックな指標で理論的な根拠はあまりありませんが、重要語を上手く抽出でき、文書を特徴づけることができることが知られています。

そのため、頻度の代わりにtf-idfを要素としてテキストデータをベクトル化することがよく行われています。